In [5]:
/*
 * Si Python et/ou R ne fonctionnent pas, voyez: https://github.com/dotnet/interactive/blob/HEAD/docs/jupyter-in-polyglot-notebooks.md
 *
 * Enlevez les commentaires qui suivent afin de connecter Python et R.
 */

// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3

// #!connect jupyter --kernel-name Rkernel --kernel-spec ir

In [28]:
#r "nuget: Microsoft.Extensions.Http, 8.0.0"

## Détermination des dates du premier et dernier circuit

Afin de sélectionner alléatoirement des circuits, il est necéssaire de savoir l'ensemble des circuits possibles. À ce fin, puisque l'API de TMX n'expose aucune méthode de choix alléatoire de circuit, la date de publication de chaque circuit sera utilisée.

In [29]:
using System;
using System.Globalization;
using System.Net.Http;
using System.Text.Json;

#nullable enable

public class TMXInputs
{
  public class MapInputsParameters {
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[] fields { get; set; } = new String[1]{"TrackId"};
      /// <summary>
      /// Search Order
      /// </summary>
    public Order1? order1 { get; set; }
      /// <summary>
      /// Amount of results (default: 40)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified TrackId and given order - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified TrackId and given order - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified TrackId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// Filter by TrackId's
      /// </summary>
    public Int64[,]? id { get; set; }
      /// <summary>
      /// Filter by UId or Track file signatures
      /// </summary>
    public String[,]? uid { get; set; }
      /// <summary>
      /// Filter by Tags[]
      /// </summary>
    public Tag[]? tag { get; set; }
      /// <summary>
      /// Filter by excluding Tags[]
      /// </summary>
    public Tag[]? etag { get; set; }
      /// <summary>
      /// true: track must have all specified tags, false: track must have one of the specified tags
      /// </summary>
    public readonly Boolean taginclusive = true;
      /// <summary>
      /// Filter by PrimaryType
      /// </summary>
    public readonly PrimaryType primarytype = PrimaryType.Race;
      /// <summary>
      /// Filter by min. UploadedAt
      /// </summary>
    public DateTime? uploadedafter { get; set; }
      /// <summary>
      /// Filter by max. UploadedAt
      /// </summary>
    public DateTime? uploadedbefore { get; set; }
      /// <summary>
      /// Filter by min. AuthorTime in ms
      /// </summary>
    public Int32? authortimemin = 30000;
      /// <summary>
      /// Filter by max. AuthorTime in ms
      /// </summary>
    public Int32? authortimemax = 120000;
      /// <summary>
      /// Filter by Routes
      /// </summary>
    public readonly Routes routes = Routes.Single;
      /// <summary>
      /// 1: Track has at least one replay
      /// </summary>
    public readonly Int32 inhasrecord = 1;
      /// <summary>
      /// 1: Track is environment mixed (Environment =/= Car)
      /// </summary>
    public readonly Int32 inenvmix = 0;
      /// <summary>
      /// 1: Track requires to be played with TMUnlimiter, 0: vanilla game
      /// </summary>
    public readonly Int32 inunlimiter = 0;

    public MapInputsParameters() {}

    public MapInputsParameters(String[]? fields, Order1? order1, Int32? count, Int64? after, Int64? before, Int64? from, Int64[,]? id, String[,]? uid, Tag? tag, Tag? etag, DateTime? uploadedafter, DateTime? uploadedbefore)
    {

    }

    public override string ToString(){
      // Voir https://stackoverflow.com/questions/4276566/how-can-you-loop-over-the-properties-of-a-class
      var props = this.GetType().GetProperties();

      string paramaterString = $"fields={String.Join("%2C", fields)}";

      foreach(var prop in props) {
        object? value = prop.GetValue(this, null);
        if (value == null || prop.Name == "fields") { continue; }

        // If DateTime
        if (value is DateTime) {
          string dateString = ((DateTime) value).ToString("s", System.Globalization.CultureInfo.InvariantCulture);

          paramaterString += $"&{prop.Name}={dateString}";
          continue;
        }

        // If Enum[]
        if (value is Enum[]) {
          paramaterString += $"&{prop.Name}=";

          var enumValue = (Enum[]) value;

          paramaterString += enumValue[0].ToString("D");

          for (int i = 1; i < enumValue.Length; i++) {
            paramaterString += $"%2C{enumValue[0].ToString("D")}";
          }
          
          continue;
        }

        // If Enum
        if (value is Enum) {
          paramaterString += $"&{prop.Name}={((Enum) value).ToString("D")}";
          continue;
        }

        paramaterString += $"&{prop.Name}={value}";
      }
      
      return paramaterString;
    }  
  }

  public enum Order1 {
    None = 0,
    UploadDateOldest = 1,
    UploadDateNewestDefault = 2,
    UpdateDateOldest = 3,
    UpdateDateNewest = 4,
    AwardCountAscending = 5,
    AwardCountDescending = 6,
    CommentCountAscending = 7,
    CommentCountDescending = 8,
    ActivityOldest = 9,
    ActivityNewest = 10,
    TrackNameAZ = 11,
    TrackNameZA = 12,
    AuthorNameAZ = 13,
    AuthorNameZA = 14,
    DifficultyAscending = 15,
    DifficultyDescending = 16,
    DownloadCountAscending = 17,
    DownloadCountDescending = 18,
    TrackValueAscending = 19,
    TrackValueDescending = 20,
    AwardsthisweekAscending = 21,
    AwardsthisweekDescending = 22,
    AwardsthismonthAscending = 23,
    AwardsthismonthDescending = 24,
    LastAwardDateOldest = 25,
    LastAwardDateNewest = 26,
    WorldRecordsetOldest = 27,
    WorldRecordsetNewest = 28,
    PersonalrecordsetOldestLoginrequired = 29,
    PersonalrecordsetNewestLoginrequired = 30,
    PersonalawardsetOldestLoginrequired = 31,
    PersonalawardsetNewestLoginrequired = 32,
    PersonaldownloadOldestLoginrequired = 33,
    PersonaldownloadNewestLoginrequired = 34,
    PersonalcommentsetOldestLoginrequired = 35,
    PersonalcommentsetNewestLoginrequired = 36,
    UserRecordPositionLowest = 37,
    UserRecordPositionHighest = 38,
  }

  public enum Tag {
    Normal = 0,
    //Stunt = 1,
    //Maze = 2,
    Offroad = 3,
    //Laps = 4,
    Fullspeed = 5,
    //LOL = 6,
    Tech = 7,
    SpeedTech = 8,
    //RPG = 9,
    PressForward = 10,
    //Trial = 11,
    Grass = 12,
  }

  public enum PrimaryType {
    Race = 0,
    Puzzle = 1,
    Platform = 2,
    Stunts = 3,
    Shortcut = 4,
    Laps = 5,
  }

  public enum LBType {
    Allreplaytypes = -1,
    Normal = 0,
    Classic = 1,
    Nadeo = 2,
    Useless = 3,
  }

  public enum Difficulty {
    Beginner = 0,
    Intermediate = 1,
    Expert = 2,
    Lunatic = 3,
  }

  public enum Routes {
    Single = 0,
    Multiple = 1,
    Symmetrical = 2,
  }
}

public class TMXResponses
{
  public static DateTime newestMapAt { get; set; }
  public static DateTime oldestMapAt { get; set; }

  public class ParsedResponse<T>
  {
    public Boolean More { get; set; }
    public T[]? Results { get; set; }
  }

  public class MapResponse
  {
      public int? TrackId { get; set; }
      public string? TrackName { get; set; }
      public string? UId { get; set; }
      public int? AuthorTime { get; set; }
      public Uploader? Uploader { get; set; }
      public DateTime? UploadedAt { get; set; }
      public DateTime? UpdatedAt { get; set; }
      public int? PrimaryType { get; set; } // enum
      public int? TrackValue { get; set; }
      public int? Style { get; set; } // enum
      public int? Routes { get; set; } // enum
      public int? Difficulty { get; set; }
      public int? Environment { get; set; } // enum
      public int? Car { get; set; } // enum
      public WRReplay? WRReplay { get; set; }
  }

  public class Uploader
  {
    public int? UserId { get; set; }
    public string? Name { get; set; }
  }

  public class WRReplay
  {
    public int ReplayId { get; set; }
    public int ReplayTime { get; set; }
  }
}

public class TMXAPI
{
  private static HttpClient sharedClient = new()
  {
      BaseAddress = new Uri("https://tmnf.exchange"),
  };

  private static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponse(TMXInputs.MapInputsParameters paramaters)
  {
    using HttpResponseMessage response = await sharedClient.GetAsync($"api/tracks?{paramaters}"); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine($"Get: api/tracks?{paramaters}");
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

    return mapResponse;
  }

  public static async Task NewestTrack()
  {
      TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = await getMapResponse(new TMXInputs.MapInputsParameters{fields=["UploadedAt"], count = 1});

      TMXResponses.newestMapAt = mapResponse?.Results?[0].UploadedAt??new DateTime();
  }

  public static async Task OldestTrack()
  {
      TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = await getMapResponse(new TMXInputs.MapInputsParameters{fields=["UploadedAt"], count = 1, order1 = TMXInputs.Order1.UploadDateOldest});

      TMXResponses.oldestMapAt = mapResponse?.Results?[0].UploadedAt??new DateTime();
  }

  public static async Task<int> GetMapIdFromDate(DateTime baseDate)
  {
      TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = await getMapResponse(new TMXInputs.MapInputsParameters{fields=["TrackId"], count = 1, uploadedbefore = baseDate});

      if (mapResponse?.Results?.Length < 1) {
        mapResponse = await getMapResponse(new TMXInputs.MapInputsParameters{fields=["TrackId"], count = 1, uploadedafter = baseDate});
      }

      if (mapResponse == null) { throw new InvalidOperationException("No map was found meeting the criteria."); }

      return mapResponse?.Results?[0]?.TrackId??0; // An ID of 0 is invalid
  }

  // public static async Task GetAsync(HttpClient httpClient)
  // {
  //     using HttpResponseMessage response = await httpClient.GetAsync("api/tracks?fields=TrackId%2CUploader.UserId&2CUploader.Name%2CAuthors%5B%5D&tag=Normal");
      
  //     response.EnsureSuccessStatusCode();
      
  //     var jsonResponse = await response.Content.ReadAsStringAsync();
  //     Console.WriteLine($"{jsonResponse}\n");
  // }
}

await TMXAPI.NewestTrack();
await TMXAPI.OldestTrack();

Console.WriteLine($"Newest map upload: {TMXResponses.newestMapAt}");
Console.WriteLine($"Oldest map upload: {TMXResponses.oldestMapAt}");


## Sélection pseudo-alléatoire de circuits

Pour sélectionner un circuit alléatoire, nous choisissons une date alléatoire entre la date de publication du plus ancient et du plus nouveau circuit.

In [92]:
if (TMXResponses.newestMapAt == null || TMXResponses.oldestMapAt == null) { return; }

public class RandomMap()
{
  static readonly TimeSpan dateDif = TMXResponses.newestMapAt - TMXResponses.oldestMapAt;
  
  public static async Task selectMapId() {
    Random random = new Random();
    Double randomDouble = random.NextDouble();

    TimeSpan randomDateDif = dateDif.Multiply(randomDouble);

    DateTime randomDate = TMXResponses.oldestMapAt.Add(randomDateDif);

    Console.WriteLine(randomDate);

    int chosenId = await TMXAPI.GetMapIdFromDate(randomDate, new TMXInputs.MapInputsParameters{});

    Console.WriteLine(chosenId);
  }
}


await RandomMap.selectMapId();